In [128]:
from gym.spaces import Dict, Discrete, MultiDiscrete, Tuple, Box
import numpy as np

from ray.rllib.env.multi_agent_env import MultiAgentEnv, ENV_STATE

# Environment

In [129]:
class FoodGame(MultiAgentEnv):
    def __init__(self, env_config):
        # Game Settings
        self.agents = env_config['agents'] # [0,1,...]
        self.agent_names = ['agent'+str(i) for i in range(len(self.agents))]
        self.food_loc = env_config['food_loc'] # {'food1': (row,col), ... } in 0 index
        self.agent_loc = env_config['agent_loc'] # {0: (row,col), 1: (row,col), ...} in 0 index
        self.board_size = env_config['board_size'] # (rows,cols)
        self.time_limit = 20
        self.cur_timestep = 0
        self.mode = env_config['mode']
        self.trainer = env_config['trainer']

        # Environment Settings
        self.action_space = Discrete(4) # L,R,U,D
        if self.trainer == 'maddpg':
            self.observation_space = Box(low=np.array([0, 0]*len(self.agents)+[0,0]*len(self.food_loc.items())), high=np.array([self.board_size[0]-1, self.board_size[1]-1]*len(self.agents)+[self.board_size[0], self.board_size[1]]*len(self.food_loc.items())), dtype=np.float32)
        else:
            self.observation_space = MultiDiscrete([self.board_size[0], self.board_size[1]]*len(self.agents)+[1+self.board_size[0], 1+self.board_size[1]]*len(self.food_loc.items())) # agent1_loc, agent2_loc, food1_loc, ...
        self.state = None

    def reset(self):
        s = ()
        self.cur_timestep = 0
        for key in self.agent_loc:
            s = s + self.agent_loc[key]

        for key in self.food_loc:
            s = s + (self.food_loc[key][0]+1,self.food_loc[key][1]+1)

        self.state = np.array(s)
#         if self.trainer == "qmix":
#             return {agent: {"obs": self.state.copy(), ENV_STATE: self.state.copy()} for agent in self.agent_names}
        return {agent: self.state.copy() for agent in self.agent_names}

    def render(self):
        board = np.full(self.board_size,'',dtype=object)
        s = self.state.reshape(-1,2)
        agents_loc = s[:len(self.agents),:]
        foods_loc = s[len(self.agents):,:]
        for i in range(len(agents_loc)):
            board[agents_loc[i][0],agents_loc[i][1]] += str(i)
        for j in range(len(foods_loc)):
            if tuple(foods_loc[j]) != (0,0):
                board[foods_loc[j][0]-1,foods_loc[j][1]-1] += 'F'
        print(board)
        print()
        return board

    def step(self, action_dict):
        self.cur_timestep += 1
        '''
        actions = {"agent1": ..., "agent2": ..., ...}
        '''
        if self.trainer == 'maddpg':
            action_dict = {
                    k: np.random.choice([0, 1, 2, 3], p=v) for k, v in action_dict.items()
                }
        s = self.state.reshape(-1,2)
        rewards = {}
        for k in self.agent_names:
            rewards[k] = -1 # penalty per step
        food_loc_lt = s[len(self.agent_names):,:]
        for i in range(len(self.agent_names)):
            action = action_dict[self.agent_names[i]]
            if action == 0:
                # L
                new_pos = [s[i][0], max(0,s[i][1]-1)]
            elif action == 1:
                # R
                new_pos = [s[i][0], min(self.board_size[1]-1,s[i][1]+1)]
            elif action == 2:
                # U
                new_pos = [max(0,s[i][0]-1), s[i][1]]
            elif action == 3:
                # D
                new_pos = [min(self.board_size[0]-1,s[i][0]+1), s[i][1]]
            else:
                raise ValueError('ActionError')

            s[i] = np.array(new_pos)
            for j in range(len(food_loc_lt)):
                if new_pos[0] == food_loc_lt[j][0]-1 and new_pos[1] == food_loc_lt[j][1]-1:
                    s[j+len(self.agent_names)] = np.array([0,0])
                    if self.mode == "coop":
                        for k in self.agent_names:
                            rewards[k] += 10 # cooperative global reward
                    else:
                        rewards[self.agent_names[i]] += 10 # competitive individual reward
        new_obs_single = s.flatten()
        done = True
        for j in range(len(food_loc_lt)):
            if tuple(food_loc_lt[j]) != (0,0):
                done = False
                break
        if done or self.time_limit == self.cur_timestep:
            dones = {agent: True for agent in self.agent_names}
            dones['__all__'] = True
        else:
            dones = {agent: False for agent in self.agent_names}
            dones['__all__'] = False
            
        infos = {agent: dict() for agent in self.agent_names}

#         if self.trainer == 'qmix':
#             new_obs = {agent: {"obs": new_obs_single.copy(), ENV_STATE: new_obs_single.copy()} for agent in self.agent_names}
#         else:
        new_obs = {agent: new_obs_single.copy() for agent in self.agent_names}

        return new_obs, rewards, dones, infos

In [130]:
def simulate_environment():
    env_config = dict()
    env_config['agents'] = [0,1]
    env_config['food_loc'] = {'food1': (3,3), 'food2': (5,5)}
    env_config['agent_loc'] = {0: (4,4), 1: (6,7)}
    env_config['board_size'] = (7,10)
    env_config['mode'] = "comp"
    env_config['trainer'] = 'ppo'

    fg = FoodGame(env_config)
    fg.reset()
    fg.render()
    MOVE_MAPPING = {0: 'L', 1: 'R', 2: 'U', 3: 'D'}
    cumu_rewards = [0,0]
    for i in range(100):
        a0_action = np.random.randint(4)
        a1_action = np.random.randint(4)
        print('Actions: agent0 = {}, agent1 = {}'.format(MOVE_MAPPING[a0_action],MOVE_MAPPING[a1_action]))
        a, rewards, dones, infos = fg.step(action_dict={'agent0': a0_action, 'agent1': a1_action})
        fg.render()
        cumu_rewards[0] += rewards['agent0']
        cumu_rewards[1] += rewards['agent1']
        if (np.array(list(dones.values())) == True).all():
            print("Episode Ended")
            print("Rewards:",cumu_rewards)
            break

from ray.tune.registry import register_env

register_env("foodenv", FoodGame)  

In [14]:
simulate_environment()

[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Actions: agent0 = D, agent1 = D
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '0' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Actions: agent0 = R, agent1 = L
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' '' '1' '' '' '']]

Actions: agent0 = U, agent1 = R
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' '' '' '' '

# Model (not used)

In [50]:
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC

torch, nn = try_import_torch()

class TorchCustomModel(TorchModelV2, nn.Module):
    """Example of a PyTorch custom model that just delegates to a fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(
            self, obs_space, action_space, num_outputs, model_config, name
        )
        nn.Module.__init__(self)

        self.torch_sub_model = TorchFC(
            obs_space, action_space, num_outputs, model_config, name
        )

    def forward(self, input_dict, state, seq_lens):
        input_dict["obs"] = input_dict["obs"].float()
        fc_out, _ = self.torch_sub_model(input_dict, state, seq_lens)
        return fc_out, []

    def value_function(self):
        return torch.reshape(self.torch_sub_model.value_function(), [-1])
    
from ray.rllib.models import ModelCatalog
ModelCatalog.register_custom_model(
        "my_model", TorchCustomModel
    )


# Environment Config

In [131]:
# Coop Environment
coop_env_config = dict()
coop_env_config['agents'] = [0,1]
coop_env_config['food_loc'] = {'food1': (3,3), 'food2': (5,5)}
coop_env_config['agent_loc'] = {0: (4,4), 1: (6,7)}
coop_env_config['board_size'] = (7,10)
coop_env_config['mode'] = 'coop'
coop_env_config['trainer'] = 'ppo'

# Competitive Environment
comp_env_config = dict()
comp_env_config['agents'] = [0,1]
comp_env_config['food_loc'] = {'food1': (3,3), 'food2': (5,5)}
comp_env_config['agent_loc'] = {0: (4,4), 1: (6,7)}
comp_env_config['board_size'] = (7,10)
comp_env_config['mode'] = 'comp'
comp_env_config['trainer'] = 'ppo'

In [132]:
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
from ray.rllib.policy.policy import PolicySpec
from ray import tune
from datetime import datetime
import os

# PPO Trainer (Coop)

In [162]:
e = FoodGame(coop_env_config)

# For custom NN models
# def gen_policy(i):
#     config = {
#         "model": {
#             "custom_model": "my_model",
#         },
#         "gamma": 0.99,
#     }
#     return (None, e.observation_space, e.action_space, config)

policies = {"policy"+str(i):  PolicySpec(
                        observation_space= e.observation_space,
                        action_space= e.action_space,
                        config={"agent_id": i}) for i in range(len(e.agents))}

policy_mapping = {'agent'+str(i): "policy"+str(i) for i in range(len(e.agents))}

def p(agent_id):
    print(agent_id)
    return policies[policy_keys[agent_id]]

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["multiagent"] = {"policies": policies,  "policy_mapping_fn": lambda agent_id: policy_mapping[agent_id]}
config["env"] = "foodenv"
config["num_workers"] = 1
config["timesteps_per_iteration"] = 1000
config["env_config"] = coop_env_config

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"training_iteration": 20}

analysis = ray.tune.run(
    ppo.PPOTrainer,
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_freq=1,
    checkpoint_at_end=True)

Trial name,status,loc
PPOTrainer_foodenv_d9b3e_00000,PENDING,


(PPOTrainer pid=4233) 2022-03-06 02:54:56,052	INFO trainer.py:2055 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPOTrainer pid=4233) 2022-03-06 02:54:56,053	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=4233) 2022-03-06 02:54:56,053	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=4232) 2022-03-06 02:55:00,653	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!


Trial name,status,loc
PPOTrainer_foodenv_d9b3e_00000,RUNNING,172.17.0.2:4233


(PPOTrainer pid=4233) 2022-03-06 02:55:02,090	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=4232) 2022-03-06 02:55:02,128	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!


Trial name,status,loc
PPOTrainer_foodenv_d9b3e_00000,RUNNING,172.17.0.2:4233


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_foodenv_d9b3e_00000,RUNNING,172.17.0.2:4233,19,253.339,76000,33.9834,34,30,3.00828


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_foodenv_d9b3e_00000,RUNNING,172.17.0.2:4233,19,253.339,76000,33.9834,34,30,3.00828


Result for PPOTrainer_foodenv_d9b3e_00000:
  agent_timesteps_total: 160000
  custom_metrics: {}
  date: 2022-03-06_02-59-30
  done: true
  episode_len_mean: 3.0015003750937734
  episode_media: {}
  episode_reward_max: 34.0
  episode_reward_mean: 33.996999249812454
  episode_reward_min: 30.0
  episodes_this_iter: 1333
  episodes_total: 20315
  experiment_id: 4c60d222bcb243f3a24b5f056b18be0a
  hostname: 310c4782fa76
  info:
    learner:
      policy0:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.11249999701976776
          cur_lr: 4.999999873689376e-05
          entropy: 0.5216752290725708
          entropy_coeff: 0.0
          kl: 0.00932738184928894
          model: {}
          policy_loss: -0.0026260078884661198
          total_loss: 0.000766995653975755
          vf_explained_var: 0.9998573660850525
          vf_loss: 0.0023436725605279207
      policy1:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.010546875186264515
    

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_foodenv_d9b3e_00000,TERMINATED,172.17.0.2:4233,20,266.672,80000,33.997,34,30,3.0015


(RolloutWorker pid=4232) 2022-03-06 02:59:30,711	ERROR worker.py:432 -- SystemExit was raised from the worker.
(RolloutWorker pid=4232) Traceback (most recent call last):
(RolloutWorker pid=4232)   File "python/ray/_raylet.pyx", line 636, in ray._raylet.execute_task
(RolloutWorker pid=4232)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.execute_task
(RolloutWorker pid=4232)   File "python/ray/_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=4232)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/_private/function_manager.py", line 639, in actor_method_executor
(RolloutWorker pid=4232)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=4232)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/util/tracing/tracing_helper.py", line 451, in _resume_span
(RolloutWorker pid=4232)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=4232)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/ac

# Results Analysis (Coop)

In [163]:
# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean",mode="max"),
    metric="episode_reward_mean")

best_checkpoint = sorted(checkpoints, key=lambda x: x[1])[-1]

In [165]:
trainer = ppo.PPOTrainer(config=config, env="foodenv")
trainer.restore(best_checkpoint[0])

(RolloutWorker pid=4360) 2022-03-06 03:00:39,901	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-03-06 03:00:41,327	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-03-06 03:00:41,427	INFO trainable.py:473 -- Restored on 172.17.0.2 from checkpoint: /home/ray/cs4246/06_03_2022_02:54:52/PPOTrainer_2022-03-06_02-54-52/PPOTrainer_foodenv_d9b3e_00000_0_2022-03-06_02-54-52/checkpoint_000020/checkpoint-20
2022-03-06 03:00:41,427	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 20, '_timesteps_total': 80000, '_time_total': 266.6717360019684, '_episodes_total': 20315}


In [171]:
# get weights: trainer.get_weights()

# run until episode ends
episode_reward = {'agent'+str(i):0 for i in range(len(e.agents))}
done = False
obs = e.reset()
print("Initial state at Time 0:")
e.render()
time = 1
while not done:
    print("Time:",time)
    time += 1
    action = trainer.compute_actions(obs,policy_id='policy0')
    for i in range(1,len(e.agents)):
        a = trainer.compute_actions(obs,policy_id='policy'+str(i))
        action['agent'+str(i)] = a['agent'+str(i)]
    obs, reward, dones, info = e.step(action)
    done = dones['__all__']
    e.render()
    for i in range(len(e.agents)):
        episode_reward['agent'+str(i)] += reward['agent'+str(i)]
print("Episode Ended")
print("Episode Rewards",episode_reward)

Initial state at Time 0:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 1
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '0' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '1' '' '' '']]

Time: 2
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '0' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '1' '' '' '' '']]

Time: 3
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '0' '' '' '' '' '' '']
 ['' '' '' '' '' '1' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Episod

# PPO Trainer (Comp)

In [123]:
e = FoodGame(comp_env_config)

# For custom NN models
# def gen_policy(i):
#     config = {
#         "model": {
#             "custom_model": "my_model",
#         },
#         "gamma": 0.99,
#     }
#     return (None, e.observation_space, e.action_space, config)

policies = {"policy"+str(i):  PolicySpec(
                        observation_space= e.observation_space,
                        action_space= e.action_space,
                        config={"agent_id": i}) for i in range(len(e.agents))}

policy_mapping = {'agent'+str(i): "policy"+str(i) for i in range(len(e.agents))}

def p(agent_id):
    print(agent_id)
    return policies[policy_keys[agent_id]]

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["multiagent"] = {"policies": policies,  "policy_mapping_fn": lambda agent_id: policy_mapping[agent_id]}
config["env"] = "foodenv"
config["timesteps_per_iteration"] = 1000
config["num_workers"] = 1
config["env_config"] = comp_env_config

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"training_iteration": 20}

analysis = ray.tune.run(
    ppo.PPOTrainer,
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_freq=1,
    checkpoint_at_end=True)

Trial name,status,loc
PPOTrainer_foodenv_f2dcd_00000,PENDING,


(PPOTrainer pid=3770) 2022-03-06 02:34:09,852	INFO trainer.py:2055 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPOTrainer pid=3770) 2022-03-06 02:34:09,853	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=3770) 2022-03-06 02:34:09,853	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=3769) 2022-03-06 02:34:14,457	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!


Trial name,status,loc
PPOTrainer_foodenv_f2dcd_00000,RUNNING,172.17.0.2:3770


(PPOTrainer pid=3770) 2022-03-06 02:34:15,873	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=3769) 2022-03-06 02:34:15,911	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!


Trial name,status,loc
PPOTrainer_foodenv_f2dcd_00000,RUNNING,172.17.0.2:3770


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_foodenv_f2dcd_00000,RUNNING,172.17.0.2:3770,19,239.793,76000,7.98496,8,6,6.00752


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_foodenv_f2dcd_00000,RUNNING,172.17.0.2:3770,19,239.793,76000,7.98496,8,6,6.00752


Result for PPOTrainer_foodenv_f2dcd_00000:
  agent_timesteps_total: 160000
  custom_metrics: {}
  date: 2022-03-06_02-38-29
  done: true
  episode_len_mean: 5.994011976047904
  episode_media: {}
  episode_reward_max: 14.0
  episode_reward_mean: 8.011976047904191
  episode_reward_min: 6.0
  episodes_this_iter: 668
  episodes_total: 12295
  experiment_id: 5e56b41f05ce4fff83faa7e71123084e
  hostname: 310c4782fa76
  info:
    learner:
      policy0:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.08437500149011612
          cur_lr: 4.999999873689376e-05
          entropy: 0.21561482548713684
          entropy_coeff: 0.0
          kl: 0.004171359818428755
          model: {}
          policy_loss: -0.00849121529608965
          total_loss: 0.2004651129245758
          vf_explained_var: 0.9855367541313171
          vf_loss: 0.2086043655872345
      policy1:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.08437500149011612
          cur_

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_foodenv_f2dcd_00000,TERMINATED,172.17.0.2:3770,20,252.385,80000,8.01198,14,6,5.99401


(RolloutWorker pid=3769) 2022-03-06 02:38:29,769	ERROR worker.py:432 -- SystemExit was raised from the worker.
(RolloutWorker pid=3769) Traceback (most recent call last):
(RolloutWorker pid=3769)   File "python/ray/_raylet.pyx", line 636, in ray._raylet.execute_task
(RolloutWorker pid=3769)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.execute_task
(RolloutWorker pid=3769)   File "python/ray/_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=3769)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/_private/function_manager.py", line 639, in actor_method_executor
(RolloutWorker pid=3769)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=3769)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/util/tracing/tracing_helper.py", line 451, in _resume_span
(RolloutWorker pid=3769)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=3769)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/ac

# Results Analysis (Comp)

In [124]:
# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean",mode="max"),
    metric="episode_reward_mean")

best_checkpoint = sorted(checkpoints, key=lambda x: x[1])[-1]
last_checkpoint = checkpoints[-1]

trainer = ppo.PPOTrainer(config=config, env="foodenv")
trainer.restore(last_checkpoint[0])

# get weights: trainer.get_weights()

# run until episode ends
episode_reward = {'agent'+str(i):0 for i in range(len(e.agents))}
done = False
obs = e.reset()
print("Initial state at Time 0:")
e.render()
time = 1
while not done:
    print("Time:",time)
    time += 1
    action = trainer.compute_actions(obs,policy_id='policy0')
    for i in range(1,len(e.agents)):
        a = trainer.compute_actions(obs,policy_id='policy'+str(i))
        action['agent'+str(i)] = a['agent'+str(i)]
    obs, reward, dones, info = e.step(action)
    done = dones['__all__']
    e.render()
    for i in range(len(e.agents)):
        episode_reward['agent'+str(i)] += reward['agent'+str(i)]
print("Episode Ended")
print("Episode Rewards",episode_reward)

(RolloutWorker pid=3855) 2022-03-06 02:38:46,097	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-03-06 02:38:47,481	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-03-06 02:38:47,583	INFO trainable.py:473 -- Restored on 172.17.0.2 from checkpoint: /home/ray/cs4246/06_03_2022_02:34:05/PPOTrainer_2022-03-06_02-34-05/PPOTrainer_foodenv_f2dcd_00000_0_2022-03-06_02-34-06/checkpoint_000020/checkpoint-20
2022-03-06 02:38:47,584	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 20, '_timesteps_total': 80000, '_time_total': 252.38456273078918, '_episodes_total': 12295}


Initial state at Time 0:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 1
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '1' '' '' '']]

Time: 2
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' '1' '' '' '' '']]

Time: 3
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '1' '' '' '' '' '']]

Time:

# DQN Trainer (Coop)

In [69]:
# Coop Environment
coop_env_config['trainer'] = 'dqn'

# Competitive Environment
comp_env_config['trainer'] = 'dqn'

In [113]:
import ray.rllib.agents.dqn as dqn

e = FoodGame(coop_env_config)

# For custom NN models
# def gen_policy(i):
#     config = {
#         "model": {
#             "custom_model": "my_model",
#         },
#         "gamma": 0.99,
#     }
#     return (None, e.observation_space, e.action_space, config)

policies = {"policy"+str(i):  PolicySpec(
                        observation_space= e.observation_space,
                        action_space= e.action_space,
                        config={"agent_id": i}) for i in range(len(e.agents))}

policy_mapping = {'agent'+str(i): "policy"+str(i) for i in range(len(e.agents))}

def p(agent_id):
    print(agent_id)
    return policies[policy_keys[agent_id]]

config = dqn.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["batch_mode"] = "complete_episodes"
config["multiagent"] = {"policies": policies,  "policy_mapping_fn": lambda agent_id: policy_mapping[agent_id]}
config["env"] = "foodenv"
config["num_workers"] = 1
config["env_config"] = coop_env_config

# Buffer params
config["buffer_size"] = 20000 # in batches
config["prioritized_replay"] = True

# Exploration params
config["exploration_config"] = {"type": "EpsilonGreedy", "epsilon_timesteps": 25000, "final_epsilon": 0.00}

# Learning params
config["lr"] = 1e-2
config["target_network_update_freq"] = 100

# Learning duration
config["timesteps_per_iteration"] = 100
config["learning_starts"] = 1000
config["train_batch_size"] = 32
config["rollout_fragment_length"] = 4 # no. of samples to rollout in each sample of the batch to add to buffer each timestep

# eval
# config["evaluation_interval"] = 1
# config["evaluation_duration"] = 1

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"timesteps_total": 30000}

analysis = ray.tune.run(
    dqn.DQNTrainer,
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_freq=100,
    checkpoint_at_end=True)

Trial name,status,loc
DQNTrainer_foodenv_76743_00000,PENDING,


(DQNTrainer pid=19093) 2022-04-01 04:11:26,065	INFO trainer.py:2055 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(DQNTrainer pid=19093) 2022-04-01 04:11:26,067	INFO simple_q.py:154 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQNTrainer pid=19093) 2022-04-01 04:11:26,067	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQNTrainer pid=19093) 2022-04-01 04:11:26,067	WARNING deprecation.py:46 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!
(RolloutWorker pid=19092) 20

Trial name,status,loc
DQNTrainer_foodenv_76743_00000,RUNNING,172.17.0.2:19093


(DQNTrainer pid=19093) 2022-04-01 04:11:33,772	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=19092) 2022-04-01 04:11:33,845	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!


Trial name,status,loc
DQNTrainer_foodenv_76743_00000,RUNNING,172.17.0.2:19093


...

Result for DQNTrainer_foodenv_76743_00000:
  agent_timesteps_total: 58664
  custom_metrics: {}
  date: 2022-04-01_04-14-19
  done: false
  episode_len_mean: 3.0
  episode_media: {}
  episode_reward_max: 34.0
  episode_reward_mean: 34.0
  episode_reward_min: 34.0
  episodes_this_iter: 68
  episodes_total: 4334
  experiment_id: 6af836e00d10403b9eacf3717e17bd18
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 29236
    learner:
      policy0:
        custom_metrics: {}
        learner_stats:
          cur_lr: 0.009999999776482582
          max_q: 18.05947494506836
          mean_q: 13.764782905578613
          mean_td_error: 0.17610222101211548
          min_q: 5.303815841674805
          model: {}
        td_error:
        - 2.2279796600341797
        - 0.21038436889648438
        - -1.0966205596923828
        - -0.14990615844726562
        - -1.0966205596923828
        - -0.28546905517578125
        - 0.5570449829101562
        - 0.5570449829101562
        - 0.286959648132324

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQNTrainer_foodenv_76743_00000,RUNNING,172.17.0.2:19093,120,162.27,29740,33.8,34,24,3.1


Result for DQNTrainer_foodenv_76743_00000:
  agent_timesteps_total: 60292
  custom_metrics: {}
  date: 2022-04-01_04-14-25
  done: true
  episode_len_mean: 3.01
  episode_media: {}
  episode_reward_max: 34.0
  episode_reward_mean: 33.98
  episode_reward_min: 32.0
  episodes_this_iter: 67
  episodes_total: 4599
  experiment_id: 6af836e00d10403b9eacf3717e17bd18
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 30058
    learner:
      policy0:
        custom_metrics: {}
        learner_stats:
          cur_lr: 0.009999999776482582
          max_q: 18.256772994995117
          mean_q: 11.504793167114258
          mean_td_error: 0.5185688138008118
          min_q: 0.4442082643508911
          model: {}
        td_error:
        - 3.143796443939209
        - -0.24910831451416016
        - -0.6808938980102539
        - 0.17530250549316406
        - -0.12719345092773438
        - 0.4777870178222656
        - 0.4033021926879883
        - 0.38840293884277344
        - 0.47778701782226

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQNTrainer_foodenv_76743_00000,TERMINATED,172.17.0.2:19093,122,165.434,30146,33.98,34,32,3.01


(RolloutWorker pid=19092) 2022-04-01 04:14:26,433	ERROR worker.py:432 -- SystemExit was raised from the worker.
(RolloutWorker pid=19092) Traceback (most recent call last):
(RolloutWorker pid=19092)   File "python/ray/_raylet.pyx", line 636, in ray._raylet.execute_task
(RolloutWorker pid=19092)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.execute_task
(RolloutWorker pid=19092)   File "python/ray/_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=19092)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/_private/function_manager.py", line 639, in actor_method_executor
(RolloutWorker pid=19092)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=19092)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/util/tracing/tracing_helper.py", line 451, in _resume_span
(RolloutWorker pid=19092)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=19092)   File "/home/ray/anaconda3/lib/python3.7/site-packa

# Results Analysis (Coop + DQN)

In [55]:
# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean",mode="max"),
    metric="episode_reward_mean")

best_checkpoint = sorted(checkpoints, key=lambda x: x[1])[-1]

trainer = dqn.DQNTrainer(config=config, env="foodenv")
trainer.restore(best_checkpoint[0])

# get weights: trainer.get_weights()

# run until episode ends
episode_reward = {'agent'+str(i):0 for i in range(len(e.agents))}
done = False
obs = e.reset()
print("Initial state at Time 0:")
e.render()
time = 1
while not done:
    print("Time:",time)
    time += 1
    action = trainer.compute_actions(obs,policy_id='policy0')
    for i in range(1,len(e.agents)):
        a = trainer.compute_actions(obs,policy_id='policy'+str(i))
        action['agent'+str(i)] = a['agent'+str(i)]
    obs, reward, dones, info = e.step(action)
    done = dones['__all__']
    e.render()
    for i in range(len(e.agents)):
        episode_reward['agent'+str(i)] += reward['agent'+str(i)]
print("Episode Ended")
print("Episode Rewards",episode_reward)

2022-04-01 00:36:20,237	WARNING deprecation.py:46 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!
(RolloutWorker pid=14301) 2022-04-01 00:36:26,116	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-04-01 00:36:28,342	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-04-01 00:36:28,508	INFO trainable.py:473 -- Restored on 172.17.0.2 from checkpoint: /home/ray/cs4246/01_04_2022_00:33:00/DQNTrainer_2022-04-01_00-33-00/DQNTrainer_foodenv_f5013_00000_0_2022-04-01_00-33-00/checkpoint_000100/checkpoint-100
2022-04-01 00:36:28,509	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 100, '_timesteps_total': 3200, '_time_total': 126.66100001335144, '_episodes_total': 4709}


Initial state at Time 0:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 1
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '0' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '1' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Time: 2
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '0' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Time: 3
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '0' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '1' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Time:

(RolloutWorker pid=14301) WARNING:tensorflow:From /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/utils/exploration/epsilon_greedy.py:218: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(RolloutWorker pid=14301) Instructions for updating:
(RolloutWorker pid=14301) Prefer Variable.assign which has equivalent behavior in 2.X.


# DQN Trainer (Comp)

In [114]:
import ray.rllib.agents.dqn as dqn
e = FoodGame(comp_env_config)

# For custom NN models
# def gen_policy(i):
#     config = {
#         "model": {
#             "custom_model": "my_model",
#         },
#         "gamma": 0.99,
#     }
#     return (None, e.observation_space, e.action_space, config)

policies = {"policy"+str(i):  PolicySpec(
                        observation_space= e.observation_space,
                        action_space= e.action_space,
                        config={"agent_id": i}) for i in range(len(e.agents))}

policy_mapping = {'agent'+str(i): "policy"+str(i) for i in range(len(e.agents))}

def p(agent_id):
    print(agent_id)
    return policies[policy_keys[agent_id]]

config = dqn.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["batch_mode"] = "complete_episodes"
config["multiagent"] = {"policies": policies,  "policy_mapping_fn": lambda agent_id: policy_mapping[agent_id]}
config["env"] = "foodenv"
config["num_workers"] = 1
config["env_config"] = comp_env_config

# Buffer params
config["buffer_size"] = 20000 # in batches
config["prioritized_replay"] = False

# Exploration params
config["exploration_config"] = {"type": "EpsilonGreedy", "epsilon_timesteps": 25000, "final_epsilon": 0.00}

# Learning params
config["lr"] = 1e-2
config["target_network_update_freq"] = 100

# Learning duration
config["timesteps_per_iteration"] = 100
config["learning_starts"] = 1000
config["train_batch_size"] = 32
config["rollout_fragment_length"] = 4 # no. of samples to rollout in each sample of the batch to add to buffer each timestep

# eval
# config["evaluation_interval"] = 1
# config["evaluation_duration"] = 1

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"timesteps_total": 30000}

analysis = ray.tune.run(
    dqn.DQNTrainer,
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_freq=100,
    checkpoint_at_end=True)

Trial name,status,loc
DQNTrainer_foodenv_f3a64_00000,PENDING,


(DQNTrainer pid=20461) 2022-04-01 04:14:57,078	INFO trainer.py:2055 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(DQNTrainer pid=20461) 2022-04-01 04:14:57,079	INFO simple_q.py:154 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQNTrainer pid=20461) 2022-04-01 04:14:57,079	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQNTrainer pid=20461) 2022-04-01 04:14:57,079	WARNING deprecation.py:46 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!
(RolloutWorker pid=20460) 20

Trial name,status,loc
DQNTrainer_foodenv_f3a64_00000,RUNNING,172.17.0.2:20461


(DQNTrainer pid=20461) 2022-04-01 04:15:06,204	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=20460) 2022-04-01 04:15:06,291	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!


Trial name,status,loc
DQNTrainer_foodenv_f3a64_00000,RUNNING,172.17.0.2:20461


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQNTrainer_foodenv_f3a64_00000,RUNNING,172.17.0.2:20461,95,116.2,29703,-40,-40,-40,20


Result for DQNTrainer_foodenv_f3a64_00000:
  agent_timesteps_total: 60006
  custom_metrics: {}
  date: 2022-04-01_04-17-09
  done: true
  episode_len_mean: 20.0
  episode_media: {}
  episode_reward_max: -40.0
  episode_reward_mean: -40.0
  episode_reward_min: -40.0
  episodes_this_iter: 15
  episodes_total: 1588
  experiment_id: 43f8d33789234291abbd5b44824e2889
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 29903
    learner:
      policy0:
        custom_metrics: {}
        learner_stats:
          cur_lr: 0.009999999776482582
          max_q: -71.06665802001953
          mean_q: -74.20606231689453
          mean_td_error: -2.535921573638916
          min_q: -83.14420318603516
          model: {}
        td_error:
        - -0.07349395751953125
        - 0.25847625732421875
        - -0.26969146728515625
        - -0.45507049560546875
        - -1.0364456176757812
        - -1.1384201049804688
        - -0.08036041259765625
        - 0.32134246826171875
        - 0.027549

(DQNTrainer pid=20461) 2022-04-01 04:17:09,113	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQNTrainer_foodenv_f3a64_00000,TERMINATED,172.17.0.2:20461,96,117.54,30003,-40,-40,-40,20


2022-04-01 04:17:09,965	INFO tune.py:636 -- Total run time: 137.78 seconds (137.14 seconds for the tuning loop).


# Results Analysis (Comp + DQN)

In [47]:
# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean",mode="max"),
    metric="episode_reward_mean")

best_checkpoint = sorted(checkpoints, key=lambda x: x[1])[-1]
last_checkpoint = checkpoints[-1]

trainer = dqn.DQNTrainer(config=config, env="foodenv")
trainer.restore(last_checkpoint[0])

# get weights: trainer.get_weights()

# run until episode ends
episode_reward = {'agent'+str(i):0 for i in range(len(e.agents))}
done = False
obs = e.reset()
print("Initial state at Time 0:")
e.render()
time = 1
while not done:
    print("Time:",time)
    time += 1
    action = trainer.compute_actions(obs,policy_id='policy0')
    for i in range(1,len(e.agents)):
        a = trainer.compute_actions(obs,policy_id='policy'+str(i))
        action['agent'+str(i)] = a['agent'+str(i)]
    obs, reward, dones, info = e.step(action)
    done = dones['__all__']
    e.render()
    for i in range(len(e.agents)):
        episode_reward['agent'+str(i)] += reward['agent'+str(i)]
print("Episode Ended")
print("Episode Rewards",episode_reward)

2022-04-01 00:04:43,375	WARNING deprecation.py:46 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!
(RolloutWorker pid=13858) 2022-04-01 00:04:49,704	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-04-01 00:04:52,014	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-04-01 00:04:52,177	INFO trainable.py:473 -- Restored on 172.17.0.2 from checkpoint: /home/ray/cs4246/31_03_2022_23:50:10/DQNTrainer_2022-03-31_23-50-10/DQNTrainer_foodenv_f928e_00000_0_2022-03-31_23-50-10/checkpoint_000100/checkpoint-100
2022-04-01 00:04:52,178	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 100, '_timesteps_total': 3200, '_time_total': 121.95173215866089, '_episodes_total': 2526}


Initial state at Time 0:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 1
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' 'F' '' '1' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Time: 2
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 3
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '1' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Time:

(RolloutWorker pid=13858) WARNING:tensorflow:From /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/utils/exploration/epsilon_greedy.py:218: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(RolloutWorker pid=13858) Instructions for updating:
(RolloutWorker pid=13858) Prefer Variable.assign which has equivalent behavior in 2.X.


# MADDPG Trainer (Coop)

In [127]:
coop_env_config['trainer'] = 'maddpg'
comp_env_config['trainer'] = 'maddpg'
e = FoodGame(coop_env_config)

# For custom NN models
# def gen_policy(i):
#     config = {
#         "model": {
#             "custom_model": "my_model",
#         },
#         "gamma": 0.99,
#     }
#     return (None, e.observation_space, e.action_space, config)

policies = {"policy"+str(i):  PolicySpec(
                        observation_space= e.observation_space,
                        action_space= e.action_space,
                        config={"agent_id": i}) for i in range(len(e.agents))}

policy_mapping = {'agent'+str(i): "policy"+str(i) for i in range(len(e.agents))}

def p(agent_id):
    print(agent_id)
    return policies[policy_keys[agent_id]]

config = dict()
config["num_gpus"] = 0
config["multiagent"] = {"policies": policies,  "policy_mapping_fn": lambda agent_id: policy_mapping[agent_id]}
config["env"] = "foodenv"
config["num_workers"] = 1
config["env_config"] = coop_env_config
config["batch_mode"] = "complete_episodes"

# model
config["actor_hiddens"] = [256, 256]
config["actor_hidden_activation"] = "tanh"
config["critic_hiddens"] = [256, 256]
config["critic_hidden_activation"] = "tanh"
config["learning_starts"] = 1000 # in terms of samples
config["critic_lr"] = 1e-2 # in terms of samples
config["actor_lr"] = 1e-2 # in terms of samples

# Buffer params
config["buffer_size"] = 20000 # in batches

# Exploration params
config["exploration_config"] = {"type": "EpsilonGreedy", "epsilon_timesteps": 25000, "final_epsilon": 0.00}

# Learning params
config["lr"] = 1e-2
config["target_network_update_freq"] = 100

# Learning duration
config["timesteps_per_iteration"] = 100
config["learning_starts"] = 1000
config["train_batch_size"] = 32
config["rollout_fragment_length"] = 4 # no. of samples to rollout in each sample of the batch to add to buffer each timestep

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"timesteps_total": 30000}

analysis = ray.tune.run(
    "contrib/MADDPG",
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_freq=100,
    checkpoint_at_end=True)

Trial name,status,loc
contrib_MADDPG_foodenv_30cc8_00000,PENDING,


(MADDPGTrainer pid=27071) 2022-04-01 05:28:13,922	INFO trainer.py:2055 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(MADDPGTrainer pid=27071) 2022-04-01 05:28:13,923	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(MADDPGTrainer pid=27071) 2022-04-01 05:28:13,923	WARNING deprecation.py:46 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!
(RolloutWorker pid=27070) /home/ray/anaconda3/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=27070)   "Box bound precision lowered by casting to {}".format(sel

Trial name,status,loc
contrib_MADDPG_foodenv_30cc8_00000,RUNNING,172.17.0.2:27071


(MADDPGTrainer pid=27071) 2022-04-01 05:28:21,521	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=27070) 2022-04-01 05:28:21,573	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!
(MADDPGTrainer pid=27071) 2022-04-01 05:28:21,682	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatchBuilder` has been deprecated. Use `a child class of `SampleCollector`` instead. This will raise an error in the future!


Trial name,status,loc
contrib_MADDPG_foodenv_30cc8_00000,RUNNING,172.17.0.2:27071


...

Result for contrib_MADDPG_foodenv_30cc8_00000:
  agent_timesteps_total: 56442
  custom_metrics: {}
  date: 2022-04-01_05-29-57
  done: false
  episode_len_mean: 17.98
  episode_media: {}
  episode_reward_max: 30.0
  episode_reward_mean: -15.16
  episode_reward_min: -40.0
  episodes_this_iter: 6
  episodes_total: 1540
  experiment_id: 987af3d6744e4009b4bc9d92518314c0
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 28167
    learner:
      policy0:
        learner_stats: {}
      policy1:
        learner_stats: {}
    num_agent_steps_sampled: 56442
    num_agent_steps_trained: 94720
    num_steps_sampled: 28221
    num_steps_trained: 47360
    num_steps_trained_this_iter: 32
    num_target_updates: 241
  iterations_since_restore: 256
  node_ip: 172.17.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 15.3
    ram_util_percent: 49.1
  pid: 27071
  policy_reward_max:
    policy0: 15.0
    policy1: 15.0
  policy_reward_mean:
    policy0: -7.58

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
contrib_MADDPG_foodenv_30cc8_00000,RUNNING,172.17.0.2:27071,267,93.1012,29395,-16.38,30,-40,18.29


Result for contrib_MADDPG_foodenv_30cc8_00000:
  agent_timesteps_total: 59428
  custom_metrics: {}
  date: 2022-04-01_05-30-03
  done: false
  episode_len_mean: 18.32
  episode_media: {}
  episode_reward_max: 34.0
  episode_reward_mean: -16.64
  episode_reward_min: -40.0
  episodes_this_iter: 6
  episodes_total: 1621
  experiment_id: 987af3d6744e4009b4bc9d92518314c0
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 29651
    learner:
      policy0:
        learner_stats: {}
      policy1:
        learner_stats: {}
    num_agent_steps_sampled: 59428
    num_agent_steps_trained: 99840
    num_steps_sampled: 29714
    num_steps_trained: 49920
    num_steps_trained_this_iter: 32
    num_target_updates: 254
  iterations_since_restore: 270
  node_ip: 172.17.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf: {}
  pid: 27071
  policy_reward_max:
    policy0: 17.0
    policy1: 17.0
  policy_reward_mean:
    policy0: -8.32
    policy1: -8.32
  policy_reward_min:
    policy

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
contrib_MADDPG_foodenv_30cc8_00000,TERMINATED,172.17.0.2:27071,273,95.1373,30025,-16.56,34,-40,18.38


2022-04-01 05:30:04,765	INFO tune.py:636 -- Total run time: 115.02 seconds (114.58 seconds for the tuning loop).


# Results Analysis (Coop + MADDPG)

In [49]:
import ray.rllib.contrib.maddpg as maddpg

# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean",mode="max"),
    metric="episode_reward_mean")

best_checkpoint = sorted(checkpoints, key=lambda x: x[1])[-1]
last_checkpoint = checkpoints[-1]

trainer = maddpg.MADDPGTrainer(config=config, env="foodenv")
trainer.restore(last_checkpoint[0])

# get weights: trainer.get_weights()

# run until episode ends
episode_reward = {'agent'+str(i):0 for i in range(len(e.agents))}
done = False
obs = e.reset()
print("Initial state at Time 0:")
e.render()
time = 1
while not done:
    print("Time:",time)
    time += 1
    action = trainer.compute_actions(obs,policy_id='policy0')
    for i in range(1,len(e.agents)):
        a = trainer.compute_actions(obs,policy_id='policy'+str(i))
        action['agent'+str(i)] = a['agent'+str(i)]
    obs, reward, dones, info = e.step(action)
    done = dones['__all__']
    e.render()
    for i in range(len(e.agents)):
        episode_reward['agent'+str(i)] += reward['agent'+str(i)]
print("Episode Ended")
print("Episode Rewards",episode_reward)

(RolloutWorker pid=2860) /home/ray/anaconda3/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=2860)   "Box bound precision lowered by casting to {}".format(self.dtype)
(RolloutWorker pid=2860) /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/contrib/maddpg/maddpg_policy.py:342: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
(RolloutWorker pid=2860)   out, units=hidden, activation=activation)
(RolloutWorker pid=2860) /home/ray/anaconda3/lib/python3.7/site-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
(RolloutWorker pid=2860)   return layer.apply(inputs)
(RolloutWorker pid=2860) /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/contrib/maddpg/maddpg_policy.py:344: UserWarning: `t

Initial state at Time 0:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 1
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '0' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 2
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '1' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Time: 3
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '1' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]

Tim

# MADDPG Trainer (Comp)

In [126]:
coop_env_config['trainer'] = 'maddpg'
comp_env_config['trainer'] = 'maddpg'

e = FoodGame(comp_env_config)

# For custom NN models
# def gen_policy(i):
#     config = {
#         "model": {
#             "custom_model": "my_model",
#         },
#         "gamma": 0.99,
#     }
#     return (None, e.observation_space, e.action_space, config)

policies = {"policy"+str(i):  PolicySpec(
                        observation_space= e.observation_space,
                        action_space= e.action_space,
                        config={"agent_id": i}) for i in range(len(e.agents))}

policy_mapping = {'agent'+str(i): "policy"+str(i) for i in range(len(e.agents))}

def p(agent_id):
    print(agent_id)
    return policies[policy_keys[agent_id]]

config = dict()
config["num_gpus"] = 0
config["multiagent"] = {"policies": policies,  "policy_mapping_fn": lambda agent_id: policy_mapping[agent_id]}
config["env"] = "foodenv"
config["num_workers"] = 1
config["env_config"] = coop_env_config
config["batch_mode"] = "complete_episodes"

# model
config["actor_hiddens"] = [256, 256]
config["actor_hidden_activation"] = "tanh"
config["critic_hiddens"] = [256, 256]
config["critic_hidden_activation"] = "tanh"
config["learning_starts"] = 1000 # in terms of samples
config["critic_lr"] = 1e-2 # in terms of samples
config["actor_lr"] = 1e-2 # in terms of samples

# Buffer params
config["buffer_size"] = 20000 # in batches

# Exploration params
config["exploration_config"] = {"type": "EpsilonGreedy", "epsilon_timesteps": 25000, "final_epsilon": 0.00}

# Learning params
config["lr"] = 1e-2
config["target_network_update_freq"] = 100

# Learning duration
config["timesteps_per_iteration"] = 100
config["learning_starts"] = 1000
config["train_batch_size"] = 32
config["rollout_fragment_length"] = 4 # no. of samples to rollout in each sample of the batch to add to buffer each timestep

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"timesteps_total": 30000}

analysis = ray.tune.run(
    "contrib/MADDPG",
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
#     checkpoint_freq=100,
    checkpoint_at_end=True)

Trial name,status,loc
contrib_MADDPG_foodenv_c2dbf_00000,PENDING,


(MADDPGTrainer pid=26986) 2022-04-01 05:25:09,635	INFO trainer.py:2055 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(MADDPGTrainer pid=26986) 2022-04-01 05:25:09,637	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(MADDPGTrainer pid=26986) 2022-04-01 05:25:09,637	WARNING deprecation.py:46 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!
(RolloutWorker pid=26985) /home/ray/anaconda3/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=26985)   "Box bound precision lowered by casting to {}".format(sel

Trial name,status,loc
contrib_MADDPG_foodenv_c2dbf_00000,RUNNING,172.17.0.2:26986


(MADDPGTrainer pid=26986) 2022-04-01 05:25:17,624	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=26985) 2022-04-01 05:25:17,679	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!
(MADDPGTrainer pid=26986) 2022-04-01 05:25:17,788	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatchBuilder` has been deprecated. Use `a child class of `SampleCollector`` instead. This will raise an error in the future!


Trial name,status,loc
contrib_MADDPG_foodenv_c2dbf_00000,RUNNING,172.17.0.2:26986


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
contrib_MADDPG_foodenv_c2dbf_00000,RUNNING,172.17.0.2:26986,267,92.0076,29003,-16.52,32,-40,18.26


Result for contrib_MADDPG_foodenv_c2dbf_00000:
  agent_timesteps_total: 59490
  custom_metrics: {}
  date: 2022-04-01_05-26-59
  done: false
  episode_len_mean: 18.46
  episode_media: {}
  episode_reward_max: 32.0
  episode_reward_mean: -16.32
  episode_reward_min: -40.0
  episodes_this_iter: 5
  episodes_total: 1601
  experiment_id: eb5006d1bbc74e1d929a5571fd0b2b83
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 29705
    learner:
      policy0:
        learner_stats: {}
      policy1:
        learner_stats: {}
    num_agent_steps_sampled: 59490
    num_agent_steps_trained: 98304
    num_steps_sampled: 29745
    num_steps_trained: 49152
    num_steps_trained_this_iter: 32
    num_target_updates: 251
  iterations_since_restore: 274
  node_ip: 172.17.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 15.2
    ram_util_percent: 49.1
  pid: 26986
  policy_reward_max:
    policy0: 16.0
    policy1: 16.0
  policy_reward_mean:
    policy0: -8.16

(MADDPGTrainer pid=26986) 2022-04-01 05:27:00,423	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
contrib_MADDPG_foodenv_c2dbf_00000,TERMINATED,172.17.0.2:26986,277,95.1727,30057,-18.12,30,-40,18.86


(RolloutWorker pid=26985) 2022-04-01 05:27:00,748	ERROR worker.py:432 -- SystemExit was raised from the worker.
(RolloutWorker pid=26985) Traceback (most recent call last):
(RolloutWorker pid=26985)   File "python/ray/_raylet.pyx", line 636, in ray._raylet.execute_task
(RolloutWorker pid=26985)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.execute_task
(RolloutWorker pid=26985)   File "python/ray/_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=26985)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/_private/function_manager.py", line 639, in actor_method_executor
(RolloutWorker pid=26985)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=26985)   File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/util/tracing/tracing_helper.py", line 451, in _resume_span
(RolloutWorker pid=26985)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=26985)   File "/home/ray/anaconda3/lib/python3.7/site-packa

# Results Analysis (Comp + MADDPG)

In [32]:
import ray.rllib.contrib.maddpg as maddpg

# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean",mode="max"),
    metric="episode_reward_mean")

best_checkpoint = sorted(checkpoints, key=lambda x: x[1])[-1]
last_checkpoint = checkpoints[-1]

trainer = maddpg.MADDPGTrainer(config=config, env="foodenv")
trainer.restore(last_checkpoint[0])

# get weights: trainer.get_weights()

# run until episode ends
episode_reward = {'agent'+str(i):0 for i in range(len(e.agents))}
done = False
obs = e.reset()
print("Initial state at Time 0:")
e.render()
time = 1
while not done:
    print("Time:",time)
    time += 1
    action = trainer.compute_actions(obs,policy_id='policy0')
    for i in range(1,len(e.agents)):
        a = trainer.compute_actions(obs,policy_id='policy'+str(i))
        action['agent'+str(i)] = a['agent'+str(i)]
    obs, reward, dones, info = e.step(action)
    done = dones['__all__']
    e.render()
    for i in range(len(e.agents)):
        episode_reward['agent'+str(i)] += reward['agent'+str(i)]
print("Episode Ended")
print("Episode Rewards",episode_reward)

(RolloutWorker pid=1858) /home/ray/anaconda3/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=1858)   "Box bound precision lowered by casting to {}".format(self.dtype)
(RolloutWorker pid=1858) /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/contrib/maddpg/maddpg_policy.py:342: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
(RolloutWorker pid=1858)   out, units=hidden, activation=activation)
(RolloutWorker pid=1858) /home/ray/anaconda3/lib/python3.7/site-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
(RolloutWorker pid=1858)   return layer.apply(inputs)
(RolloutWorker pid=1858) /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/contrib/maddpg/maddpg_policy.py:344: UserWarning: `t

Initial state at Time 0:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 1
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '' '0' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '' '1' '']]

Time: 2
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '' '0' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '' '1' '' '']]

Time: 3
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' 'F' '' '' '' '' '' '']
 ['' '' '' '0' '' '' '' '' '' '']
 ['' '' '' '' '' 'F' '' '' '' '']
 ['' '' '' '' '' '' '1' '' '' '']]

Tim

# QMIX Trainer (Coop)

In [133]:
coop_env_config['trainer'] = 'qmix'
comp_env_config['trainer'] = 'qmix'

In [123]:
e = FoodGame(coop_env_config)
tuple_obs_space = Tuple([e.observation_space for i in range(len(e.agents))])
tuple_act_space = Tuple([e.action_space for i in range(len(e.agents))])

register_env("grouped_foodenv", lambda config: FoodGame(config).with_agent_groups(
            groups={"agents": e.agent_names},
            obs_space=tuple_obs_space,
            act_space=tuple_act_space,
        ))

config = dict()
config["num_gpus"] = 0
config["env"] = "grouped_foodenv"
config["num_workers"] = 1
config["env_config"] = coop_env_config

# Learning params
config["lr"] = 1e-3
config["optim_alpha"] = 0.99
config["optim_eps"] = 0.00001
config["grad_norm_clipping"] = 10
config["target_network_update_freq"] = 100

# Buffer params
config["buffer_size"] = 20000 # in batches

# Exploration params
config["exploration_config"] = {"type": "EpsilonGreedy", "epsilon_timesteps": 25000, "final_epsilon": 0.00}

# Learning duration
config["timesteps_per_iteration"] = 100
config["learning_starts"] = 1000
config["train_batch_size"] = 32
config["rollout_fragment_length"] = 4 # no. of samples to rollout in each sample of the batch to add to buffer each timestep
config["model"]={
        "lstm_cell_size": 256,
        "max_seq_len": 20,
    }

config["mixing_embed_dim"] = 256
# eval
# config["evaluation_interval"] = 1
# config["evaluation_duration"] = 1

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"timesteps_total": 30000}
analysis = ray.tune.run(
    "QMIX",
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_freq=100,
    checkpoint_at_end=True)

Trial name,status,loc
QMIX_grouped_foodenv_5371f_00000,PENDING,


(QMixTrainer pid=26724) 2022-04-01 05:14:53,115	INFO simple_q.py:154 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(QMixTrainer pid=26724) 2022-04-01 05:14:53,115	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc
QMIX_grouped_foodenv_5371f_00000,RUNNING,172.17.0.2:26724


(QMixTrainer pid=26724) 2022-04-01 05:14:56,678	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=26723) 2022-04-01 05:14:56,651	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
(RolloutWorker pid=26723) /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/agents/qmix/qmix_policy.py:455: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811697362/work/torch/csrc/utils/tensor_numpy.cpp:189.)
(RolloutWork

Result for QMIX_grouped_foodenv_5371f_00000:
  agent_timesteps_total: 118
  custom_metrics: {}
  date: 2022-04-01_05-14-57
  done: false
  episode_len_mean: 19.666666666666668
  episode_media: {}
  episode_reward_max: 4.0
  episode_reward_mean: -22.666666666666668
  episode_reward_min: -40.0
  episodes_this_iter: 6
  episodes_total: 6
  experiment_id: d11fbc22d1c74b2682ad267be9ec3218
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 118
    learner:
      default_policy:
        learner_stats:
          grad_norm: 37.676612854003906
          loss: 10.890652656555176
          q_taken_mean: -1.9169570521304482
          target_mean: -0.15125996188113564
          td_error_abs: 2.2955468830309416
    num_agent_steps_sampled: 118
    num_steps_sampled: 118
    num_steps_trained: 236
    num_steps_trained_this_iter: 38
    num_target_updates: 1
  iterations_since_restore: 1
  node_ip: 172.17.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 25

...

Result for QMIX_grouped_foodenv_5371f_00000:
  agent_timesteps_total: 29337
  custom_metrics: {}
  date: 2022-04-01_05-18-35
  done: false
  episode_len_mean: 3.0
  episode_media: {}
  episode_reward_max: 34.0
  episode_reward_mean: 34.0
  episode_reward_min: 34.0
  episodes_this_iter: 68
  episodes_total: 4775
  experiment_id: d11fbc22d1c74b2682ad267be9ec3218
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 29283
    learner:
      default_policy:
        learner_stats:
          grad_norm: 30.223304748535156
          loss: 3.38950514793396
          q_taken_mean: 10.827490604285037
          target_mean: 11.640173709753787
          td_error_abs: 1.3852162216648911
    num_agent_steps_sampled: 29337
    num_steps_sampled: 29337
    num_steps_trained: 124167
    num_steps_trained_this_iter: 33
    num_target_updates: 275
  iterations_since_restore: 159
  node_ip: 172.17.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 15.833333333333334

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
QMIX_grouped_foodenv_5371f_00000,RUNNING,172.17.0.2:26724,161,218.812,29643,34,34,34,3


Result for QMIX_grouped_foodenv_5371f_00000:
  agent_timesteps_total: 29949
  custom_metrics: {}
  date: 2022-04-01_05-18-41
  done: false
  episode_len_mean: 3.0
  episode_media: {}
  episode_reward_max: 34.0
  episode_reward_mean: 34.0
  episode_reward_min: 34.0
  episodes_this_iter: 34
  episodes_total: 4979
  experiment_id: d11fbc22d1c74b2682ad267be9ec3218
  hostname: 1d29a0c222c3
  info:
    last_target_update_ts: 29895
    learner:
      default_policy:
        learner_stats:
          grad_norm: 14.164641380310059
          loss: 0.9975647926330566
          q_taken_mean: 12.541200183686756
          target_mean: 12.88115728469122
          td_error_abs: 0.8174606504894438
    num_agent_steps_sampled: 29949
    num_steps_sampled: 29949
    num_steps_trained: 128027
    num_steps_trained_this_iter: 42
    num_target_updates: 281
  iterations_since_restore: 163
  node_ip: 172.17.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 15.2
    ram_util

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
QMIX_grouped_foodenv_5371f_00000,TERMINATED,172.17.0.2:26724,164,223.68,30153,34,34,34,3


2022-04-01 05:18:44,048	INFO tune.py:636 -- Total run time: 235.17 seconds (234.67 seconds for the tuning loop).


# Results Analysis (QMIX + Coop)

In [39]:
import ray.rllib.agents.qmix as qmix

# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean",mode="max"),
    metric="episode_reward_mean")

best_checkpoint = sorted(checkpoints, key=lambda x: x[1])[-1]
last_checkpoint = checkpoints[-1]

trainer = qmix.QMixTrainer(config=config, env="grouped_foodenv")
trainer.restore(last_checkpoint[0])

# get weights: trainer.get_weights()

# run until episode ends

grouped_e = FoodGame(coop_env_config).with_agent_groups(
            groups={"agents": FoodGame(coop_env_config).agent_names},
            obs_space=tuple_obs_space,
            act_space=tuple_act_space,
        )

# episode_reward = {'agent'+str(i):0 for i in range(len(grouped_e.agents))}
# done = False
# obs = grouped_e.reset()
# print("Initial state at Time 0:")
# grouped_e.render()
# time = 1
# while not done:
#     print("Time:",time)
#     time += 1
#     action = trainer.compute_actions(obs,policy_id='default_policy')
#     for i in range(1,len(grouped_e.agents)):
#         a = trainer.compute_actions(obs,policy_id='default_policy')
#         action['agent'+str(i)] = a['agent'+str(i)]
#     obs, reward, dones, info = grouped_e.step(action)
#     done = dones['__all__']
#     grouped_e.render()
#     for i in range(len(grouped_e.agents)):
#         episode_reward['agent'+str(i)] += reward['agent'+str(i)]
# print("Episode Ended")
# print("Episode Rewards",episode_reward)

2022-03-06 05:20:59,817	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-03-06 05:20:59,824	INFO trainable.py:473 -- Restored on 172.17.0.2 from checkpoint: /home/ray/cs4246/06_03_2022_05:19:09/QMIX/QMIX_grouped_foodenv_01f18_00000_0_2022-03-06_05-19-09/checkpoint_000020/checkpoint-20
2022-03-06 05:20:59,825	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 20, '_timesteps_total': 818, '_time_total': 64.86543583869934, '_episodes_total': 2417}
(RolloutWorker pid=2256) 2022-03-06 05:20:59,804	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
(RolloutWorker pid=2256) /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/agents/qmix/qmix_policy.py:455: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to

# QMIX Trainer (Comp)

In [137]:
e = FoodGame(comp_env_config)
tuple_obs_space = Tuple([e.observation_space for i in range(len(e.agents))])
tuple_act_space = Tuple([e.action_space for i in range(len(e.agents))])

register_env("grouped_foodenv", lambda config: FoodGame(config).with_agent_groups(
            groups={'group'+str(i):[e.agent_names[i]] for i in range(len(e.agents))},
            obs_space=tuple_obs_space,
            act_space=tuple_act_space,
        ))

config = dict()
config["num_gpus"] = 0
config["env"] = "grouped_foodenv"
config["num_workers"] = 1
config["env_config"] = comp_env_config
config["lr"] = 1e-2
config["learning_starts"] = 1000
config["target_network_update_freq"] = 500
config["rollout_fragment_length"] = 4
config["train_batch_size"] = 32
config["timesteps_per_iteration"] = 1000
# config["exploration_config"] = {"epsilon_timesteps": 10000, }
config["exploration_config"] = {"epsilon_timesteps": 15000, "final_epsilon": 0.0}


now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H:%M:%S")
os.mkdir('./'+dt_string)
log_dir = './'+dt_string+'/'

stop_criteria = {"training_iteration": 20}

analysis = ray.tune.run(
    "QMIX",
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_freq=1,
    checkpoint_at_end=True)

Trial name,status,loc
QMIX_grouped_foodenv_8e6da_00000,PENDING,


(QMixTrainer pid=27216) 2022-04-01 05:38:00,381	INFO simple_q.py:154 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(QMixTrainer pid=27216) 2022-04-01 05:38:00,381	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc
QMIX_grouped_foodenv_8e6da_00000,RUNNING,172.17.0.2:27216


2022-04-01 05:38:03,688	ERROR trial_runner.py:927 -- Trial QMIX_grouped_foodenv_8e6da_00000: Error processing event.
Traceback (most recent call last):
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 893, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 707, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/worker.py", line 1733, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::QMixTrainer.train() (pid=27216, ip=172.17.0.2, repr=QMixTrainer)
  File "/home/ray/anaconda3/lib/python3.7/site-packages/ray/tune/trainable.py", line 315, in train
    result = self.

Result for QMIX_grouped_foodenv_8e6da_00000:
  date: 2022-04-01_05-38-03
  experiment_id: adc7f78f897e493d94465bc561c0a724
  hostname: 1d29a0c222c3
  node_ip: 172.17.0.2
  pid: 27216
  timestamp: 1648816683
  trial_id: 8e6da_00000
  


Trial name,status,loc
QMIX_grouped_foodenv_8e6da_00000,ERROR,172.17.0.2:27216
Trial name,# failures,error file
QMIX_grouped_foodenv_8e6da_00000,1,/home/ray/cs4246/01_04_2022_05:37:56/QMIX/QMIX_grouped_foodenv_8e6da_00000_0_2022-04-01_05-37-56/error.txt


(QMixTrainer pid=27216) 2022-04-01 05:38:03,670	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(QMixTrainer pid=27216) 2022-04-01 05:38:03,685	WARNING trainer.py:974 -- Worker crashed during call to `step_attempt()`. To try to continue training without the failed worker, set `ignore_worker_failures=True`.
(RolloutWorker pid=27215) 2022-04-01 05:38:03,655	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
(RolloutWorker pid=27215) /home/ray/anaconda3/lib/python3.7/site-packages/ray/rllib/agents/qmix/qmix_policy.py:455: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting 

TuneError: ('Trials did not complete', [QMIX_grouped_foodenv_8e6da_00000])

In [136]:
{'group'+str(i):e.agent_names[i] for i in range(len(e.agents))}

{'group0': 'agent0', 'group1': 'agent1'}